In [5]:
'''
Last updated: 1.28.2025

USD/SEK ("U.S. Dollar / Swedish Krona") trading strategy that focuses on the Simple Moving Average ("SMA") trend. 
20-day SMA vs 100-day SMA, over a 5-year horizon. 
Vol is taken into consideration via an adjustment of +/- 0.5* of pip value. 
'''

import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
import matplotlib
import yfinance as yf
import datetime

#Environment-specifics
initial_aum = 600000000
risk_per_trade = 600000000 * 0.01
one_lot = 100000
margin = 2.00

#Dataframes; Data
    #Timeframe:
horizon = datetime.timedelta(365*5) #5-year horizon

    #Dataframe
USDSEK_df = yf.download("USDSEK=X", datetime.date.today()-horizon,datetime.date.today())
USDSEK_Close_Price = USDSEK_df['Close']

    #Moving Averages
sma20 = USDSEK_Close_Price.rolling(20).mean()      #blue
sma100 = USDSEK_Close_Price.rolling(100).mean()    #orange

    #Calculating our ATR
highs = []
lows = []
closes = []
ticker = "USDSEK=X"
ATR_multiple = 2.00
vol_adjustment = [.95, 1.05]


for i in range(len(USDSEK_df)):
    highs.append(USDSEK_df["High"].loc[USDSEK_df['High'].index[i], "USDSEK=X"])
    lows.append(USDSEK_df["Low"].loc[USDSEK_df["Low"].index[i], "USDSEK=X"])
    closes.append(USDSEK_df["Low"].loc[USDSEK_df["Low"].index[i], "USDSEK=X"])
highs_series = pd.Series(highs)
highs_array = np.array(highs_series)
lows_series = pd.Series(lows)
lows_array = np.array(lows_series)
closes_series = pd.Series(closes)
closes_array = np.array(closes_series)

ATR = ta.ATR(highs_array, lows_array, closes_array, timeperiod = 14)

#Trading
PNLs = []
positions = {}
trade_book = []
entry = None
exit = None

def calc_PNL(entry, exit, direction):
    percent_PNL = (((exit / entry) - 1) * (1 if direction == "BUY" else -1))
    return percent_PNL + 1


for i in range(len(USDSEK_Close_Price)):
    if i >=14:
        if i >= 379:
        #if ATR is bigger, at this moment, as the past 14 day ATR
            if ATR[i] > np.mean(ATR[i-365: i]):
                #trade smaller position due to larger vol
                if (sma20.loc[sma20.index[i-1], ticker] > sma100.loc[sma100.index[i-1], ticker]) and (sma20.loc[sma20.index[i], ticker] < sma100.loc[sma100.index[i], ticker]):
                    #Calculate Pip Value
                    direction = "SELL"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

                    #Adjust Position Size due to Vol
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[1])  * ATR[i])

                    #Append to Positions Dictionary
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "SELL",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}

                    #Calculate the Trade's PNL
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

                elif (sma20.loc[sma20.index[i-1], ticker] < sma100.loc[sma100.index[i-1], ticker]) and (sma20.loc[sma20.index[i], ticker] > sma100.loc[sma100.index[i], ticker]):
                    direction = "BUY"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[1])  * ATR[i])
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "BUY",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}                    
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

            elif ATR[i] < np.mean(ATR[i-365:i]):
                if (sma20.loc[sma20.index[i-1], ticker] > sma100.loc[sma100.index[i-1], ticker]) and (sma20.loc[sma20.index[i], ticker] < sma100.loc[sma100.index[i], ticker]):
                    direction = "SELL"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[0])  * ATR[i])
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "SELL",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}                
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

                elif (sma20.loc[sma20.index[i-1], ticker] < sma100.loc[sma100.index[i-1], ticker]) and (sma20.loc[sma20.index[i], ticker] > sma100.loc[sma100.index[i], ticker]):
                    direction = "BUY"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[0])  * ATR[i])
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "BUY",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}     
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
        elif i < 379:
            count_ATR = np.count_nonzero(ATR[14:i])
        #if ATR is bigger, at this moment, as the past 14 day ATR
            if ATR[i] > np.mean(ATR[count_ATR: i]):
                #trade smaller position due to larger vol
                if (sma20.loc[sma20.index[i-1], ticker] > sma100.loc[sma100.index[i-1], ticker]) and (sma20.loc[sma20.index[i], ticker] < sma100.loc[sma100.index[i], ticker]):
                    direction = "SELL"
                    #Calculate Pip Value
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

                    #Adjust Position Size due to Vol
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[1])  * ATR[i])

                    #Append to Positions Dictionary
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "SELL",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}

                    #Calculate the Trade's PNL
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

                elif (sma20.loc[sma20.index[i-1], ticker] < sma100.loc[sma100.index[i-1], ticker]) and (sma20.loc[sma20.index[i], ticker] > sma100.loc[sma100.index[i], ticker]):
                    direction = "BUY"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[1])  * ATR[i])
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "BUY",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}                    
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

            elif ATR[i] < np.mean(ATR[count_ATR:i]):
                if (sma20.loc[sma20.index[i-1], ticker] > sma100.loc[sma100.index[i-1], ticker]) and (sma20.loc[sma20.index[i], ticker] < sma100.loc[sma100.index[i], ticker]):
                    direction = "SELL"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[0])  * ATR[i])
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "SELL",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}                
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

                elif (sma20.loc[sma20.index[i-1], ticker] < sma100.loc[sma100.index[i-1], ticker]) and (sma20.loc[sma20.index[i], ticker] > sma100.loc[sma100.index[i], ticker]):
                    direction = "BUY"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[0])  * ATR[i])
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "BUY",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}     
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

       
#Profitability
print(np.cumprod(PNLs))
    #Sharpe Ratio
#np.mean(PNLs) / np.std(PNLs)

[*********************100%***********************]  1 of 1 completed


[1.02204341 1.04992702 1.0522072  1.10509689 0.92011729 0.92127194
 0.92545916 0.95899952 0.98360504 1.00959986 1.03986037 1.05593454
 1.08178587 1.100487   1.10373205 1.11605081]


In [4]:
#11.61% cumulative PNL
np.cumprod(np.array(PNLs))[-1]

1.1160508114800944